## Starter Code for Assignment 2 - Big Data with H2O

**Logistic Regression**

Here is some starter code for Assignment 2 - Big Data with H2O.   You will still need to extend this code somewhat.  


[H2O Docs](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html)  
[http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/index.html)  

In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool

Set up some parameters for the analysis.  

In [2]:
data_path=None
all_variables=None
test_path=None
target=None
nthreads=1 
min_mem_size=6 
run_time=333
classification=False
scale=False
max_models=9    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0 

The next sections contains helper functions for automating analysis.             

In [3]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(run_id,analysis,target,run_time,classification,scale,model,balance,balance_threshold,name,nthreads,min_mem_size):
  m_data={}
  m_data['run_id'] =run_id
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [4]:
data_path='data/logistic_regression_bin_class_ad_conversion.csv'

In [5]:
data_path = os.path.join(os.path.abspath(os.curdir),data_path)

In [6]:
all_variables=None

In [7]:
run_id=alphabet(9)
# run_id to std out
print (run_id) 

FAMH0nY14


In [8]:
server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir) 

In [9]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:8309..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /Users/bear/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmp0ntd9dml
  JVM stdout: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmp0ntd9dml/h2o_bear_started_from_python.out
  JVM stderr: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmp0ntd9dml/h2o_bear_started_from_python.err
  Server is running at http://127.0.0.1:8309
Connecting to H2O server at http://127.0.0.1:8309... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.1
H2O cluster version age:,3 months and 25 days !!!
H2O cluster name:,H2O_from_python_bear_yizl2h
H2O cluster total nodes:,1
H2O cluster free memory:,5.750 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [10]:
# meta data
meta_data = set_meta_data(run_id,analysis,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,nthreads,min_mem_size)
print(meta_data)  

{'run_id': 'FAMH0nY14', 'start_time': 1538541618.5450351, 'target': None, 'max_models': 9, 'run_time': 333, 'scale': False, 'classification': False, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1538541618.54504, 'execution_time': 0.0, 'nthreads': 1, 'min_mem_size': 6, 'analysis': 0}


In [11]:
print(data_path)

/Users/bear/Downloads/AutoML/data/logistic_regression_bin_class_ad_conversion.csv


In [12]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
df.head()

ID,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
1,68.95,35,61833.9,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,3/27/2016 0:53,0
2,80.23,31,68441.9,193.77,Monitored national standardization,West Jodi,1,Nauru,4/4/2016 1:39,0
3,69.47,26,59785.9,236.5,Organic bottom-line service-desk,Davidton,0,San Marino,3/13/2016 20:35,0
4,74.15,29,54806.2,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,1/10/2016 2:31,0
5,68.37,35,73890,225.58,Robust logistical utilization,South Manuel,0,Iceland,6/3/2016 3:36,0
6,59.99,23,59761.6,226.74,Sharable client-driven software,Jamieberg,1,Norway,5/19/2016 14:30,0
7,88.91,33,53852.8,208.36,Enhanced dedicated support,Brandonstad,0,Myanmar,1/28/2016 20:59,0
8,66,48,24593.3,131.76,Reactive local challenge,Port Jefferybury,1,Australia,3/7/2016 1:40,1
9,74.53,30,68862,221.51,Configurable coherent function,West Colin,1,Grenada,4/18/2016 9:33,0
10,69.88,20,55642.3,183.82,Mandatory homogeneous architecture,Ramirezton,1,Ghana,7/11/2016 1:42,0


In [14]:
df.describe()

Rows:1000
Cols:11




,ID,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
type,int,real,int,real,real,string,string,int,enum,string,int
mins,1.0,32.6,19.0,13996.5,104.78,NaN,NaN,0.0,,NaN,0.0
mean,500.5,65.00019999999999,36.009,55000.00008,180.0001,NaN,NaN,0.481,,NaN,0.5
maxs,1000.0,91.43,61.0,79484.8,269.96,NaN,NaN,1.0,,NaN,1.0
sigma,288.8194360957494,15.85361456750021,8.78556231012592,13414.634022282358,43.9023393019801,NaN,NaN,0.49988887654046565,,NaN,0.5002501876563868
zeros,0,0,0,0,0,0,0,519,,0,500
missing,0,0,0,0,0,0,0,0,0,0,0
0,1.0,68.95,35.0,61833.9,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0.0,Tunisia,3/27/2016 0:53,0.0
1,2.0,80.23,31.0,68441.85,193.77,Monitored national standardization,West Jodi,1.0,Nauru,4/4/2016 1:39,0.0
2,3.0,69.47,26.0,59785.94,236.5,Organic bottom-line service-desk,Davidton,0.0,San Marino,3/13/2016 20:35,0.0


describe()[source]
Generate an in-depth description of this H2OFrame.

The description is a tabular print of the type, min, max, sigma, number of zeros, and number of missing elements for each H2OVec in this H2OFrame.

Returns:	None (print to stdout) 

In [15]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
  target=df.columns[-1]   
y = target

In [16]:
print(y)

Clicked on Ad


In [17]:
print(all_variables)

None


In [18]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [19]:
df.describe()

Rows:1000
Cols:11




,ID,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
type,int,real,int,real,real,string,string,int,enum,string,int
mins,1.0,32.6,19.0,13996.5,104.78,NaN,NaN,0.0,,NaN,0.0
mean,500.5,65.00019999999999,36.009,55000.00008,180.0001,NaN,NaN,0.481,,NaN,0.5
maxs,1000.0,91.43,61.0,79484.8,269.96,NaN,NaN,1.0,,NaN,1.0
sigma,288.8194360957494,15.85361456750021,8.78556231012592,13414.634022282358,43.9023393019801,NaN,NaN,0.49988887654046565,,NaN,0.5002501876563868
zeros,0,0,0,0,0,0,0,519,,0,500
missing,0,0,0,0,0,0,0,0,0,0,0
0,1.0,68.95,35.0,61833.9,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0.0,Tunisia,3/27/2016 0:53,0.0
1,2.0,80.23,31.0,68441.85,193.77,Monitored national standardization,West Jodi,1.0,Nauru,4/4/2016 1:39,0.0
2,3.0,69.47,26.0,59785.94,236.5,Organic bottom-line service-desk,Davidton,0.0,San Marino,3/13/2016 20:35,0.0


In [20]:
# independent variables

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

In [21]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [22]:
print(classification)

False


In [23]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [24]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok, val   

In [25]:
ok,val=check_y(y,df)

In [26]:
print(val)

int


In [27]:
print(y)

Clicked on Ad


In [28]:
print(ok)

True


In [29]:
if val=='enum':
    print(df[y].levels())

In [30]:
df.describe()

Rows:1000
Cols:11




,ID,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
type,int,real,int,real,real,string,string,int,enum,string,int
mins,1.0,32.6,19.0,13996.5,104.78,NaN,NaN,0.0,,NaN,0.0
mean,500.5,65.00019999999999,36.009,55000.00008,180.0001,NaN,NaN,0.481,,NaN,0.5
maxs,1000.0,91.43,61.0,79484.8,269.96,NaN,NaN,1.0,,NaN,1.0
sigma,288.8194360957494,15.85361456750021,8.78556231012592,13414.634022282358,43.9023393019801,NaN,NaN,0.49988887654046565,,NaN,0.5002501876563868
zeros,0,0,0,0,0,0,0,519,,0,500
missing,0,0,0,0,0,0,0,0,0,0,0
0,1.0,68.95,35.0,61833.9,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0.0,Tunisia,3/27/2016 0:53,0.0
1,2.0,80.23,31.0,68441.85,193.77,Monitored national standardization,West Jodi,1.0,Nauru,4/4/2016 1:39,0.0
2,3.0,69.47,26.0,59785.94,236.5,Organic bottom-line service-desk,Davidton,0.0,San Marino,3/13/2016 20:35,0.0


In [31]:
allV=get_variables_types(df)
allV

{'Ad Topic Line': 'string',
 'Age': 'int',
 'Area Income': 'real',
 'City': 'string',
 'Clicked on Ad': 'int',
 'Country': 'enum',
 'Daily Internet Usage': 'real',
 'Daily Time Spent on Site': 'real',
 'ID': 'int',
 'Male': 'int',
 'Timestamp': 'string'}

In [32]:
meta_data['variables']=allV

In [33]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.9])

In [34]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [35]:
model_start_time = time.time()

In [36]:
aml.train(x=X,y=y,training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [37]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [38]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [39]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_BestOfFamily_0_AutoML_20181003...,0.032494,0.180260,0.032494,0.095383,0.125438
1,DeepLearning_0_AutoML_20181003_004019,0.032608,0.180577,0.032608,0.098278,0.124804
2,StackedEnsemble_AllModels_0_AutoML_20181003_00...,0.033311,0.182512,0.033311,0.095788,0.128279
3,XRT_0_AutoML_20181003_004019,0.042332,0.205748,0.042332,0.098452,0.142590
4,DeepLearning_grid_0_AutoML_20181003_004019_mod...,0.043141,0.207703,0.043141,0.122975,0.148630
5,GLM_grid_0_AutoML_20181003_004019_model_0,0.043596,0.208797,0.043596,0.143951,0.141198
6,GBM_grid_0_AutoML_20181003_004019_model_58,0.044503,0.210957,0.044503,0.096704,0.150049
7,GBM_grid_0_AutoML_20181003_004019_model_88,0.045363,0.212987,0.045363,0.104667,0.150736
8,GBM_grid_0_AutoML_20181003_004019_model_113,0.046268,0.215101,0.046268,0.109142,0.151428
9,GBM_grid_0_AutoML_20181003_004019_model_51,0.046361,0.215315,0.046361,0.101372,0.152180


In [40]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [41]:
mod_best._id

'StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019'

In [42]:
# Get stacked ensemble  
se=get_stacked_ensemble(model_set)

In [43]:
print(se)

StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019


In [44]:
if se is not None:
  mod_best=h2o.get_model(se)

In [45]:
mod_best._id

'StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019'

In [46]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics>

In [47]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [48]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.020021746219559964
RMSE: 0.14149821984590466
MAE: 0.07256563196031747
RMSLE: 0.101741231001171
R^2: 0.9199107001344823
Mean Residual Deviance: 0.020021746219559964
Null degrees of freedom: 743
Residual degrees of freedom: 740
Null deviance: 185.99462365591404
Residual deviance: 14.896179187352613
AIC: -788.3560587390215

ModelMetricsRegressionGLM: stackedensemble
** Reported on validation data. **

MSE: 0.023245230221482664
RMSE: 0.15246386529759326
MAE: 0.08179697146687008
RMSLE: 0.11006320000242535
R^2: 0.9070190791140693
Mean Residual Deviance: 0.023245230221482664
Null degrees of freedom: 167
Residual degrees of freedom: 164
Null deviance: 42.00121401318072
Residual deviance: 3.905198677209088
AIC: -145.19474656281923

ModelMetricsRegressionGL

In [49]:
bm=stackedensemble_df(aml_leaderboard_df)

In [50]:
bm

['GBM_grid_0_AutoML_20181003_004019_model_58',
 'GLM_grid_0_AutoML_20181003_004019_model_0',
 'DRF_0_AutoML_20181003_004019',
 'XRT_0_AutoML_20181003_004019',
 'DeepLearning_0_AutoML_20181003_004019']

In [51]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_BestOfFamily_0_AutoML_20181003...,0.032494,0.180260,0.032494,0.095383,0.125438
1,DeepLearning_0_AutoML_20181003_004019,0.032608,0.180577,0.032608,0.098278,0.124804
2,StackedEnsemble_AllModels_0_AutoML_20181003_00...,0.033311,0.182512,0.033311,0.095788,0.128279
3,XRT_0_AutoML_20181003_004019,0.042332,0.205748,0.042332,0.098452,0.142590
4,DeepLearning_grid_0_AutoML_20181003_004019_mod...,0.043141,0.207703,0.043141,0.122975,0.148630
5,GLM_grid_0_AutoML_20181003_004019_model_0,0.043596,0.208797,0.043596,0.143951,0.141198
6,GBM_grid_0_AutoML_20181003_004019_model_58,0.044503,0.210957,0.044503,0.096704,0.150049
7,GBM_grid_0_AutoML_20181003_004019_model_88,0.045363,0.212987,0.045363,0.104667,0.150736
8,GBM_grid_0_AutoML_20181003_004019_model_113,0.046268,0.215101,0.046268,0.109142,0.151428
9,GBM_grid_0_AutoML_20181003_004019_model_51,0.046361,0.215315,0.046361,0.101372,0.152180


In [52]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [53]:
bm

['DeepLearning_0_AutoML_20181003_004019',
 'XRT_0_AutoML_20181003_004019',
 'GLM_grid_0_AutoML_20181003_004019_model_0',
 'GBM_grid_0_AutoML_20181003_004019_model_58',
 'DRF_0_AutoML_20181003_004019',
 'StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019']

In [54]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]


In [55]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_BestOfFamily_0_AutoML_20181003...,0.032494,0.180260,0.032494,0.095383,0.125438
1,DeepLearning_0_AutoML_20181003_004019,0.032608,0.180577,0.032608,0.098278,0.124804
3,XRT_0_AutoML_20181003_004019,0.042332,0.205748,0.042332,0.098452,0.142590
5,GLM_grid_0_AutoML_20181003_004019_model_0,0.043596,0.208797,0.043596,0.143951,0.141198
6,GBM_grid_0_AutoML_20181003_004019_model_58,0.044503,0.210957,0.044503,0.096704,0.150049
18,DRF_0_AutoML_20181003_004019,0.051448,0.226822,0.051448,0.131517,0.161972


In [56]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [57]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019


In [58]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019
stackedensemble


In [59]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [60]:
meta_data['models']=bm

In [61]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass    

In [62]:
print(models_path)

/Users/bear/Downloads/AutoML/FAMH0nY14/models


In [63]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass        

{'algo': 'gbm', 'model_id': 'GBM_grid_0_AutoML_20181003_004019_model_58', 'varimp': [('Daily Internet Usage', 592.63916015625, 1.0, 0.5187020078376144), ('Daily Time Spent on Site', 293.22625732421875, 0.4947804280211745, 0.2566436014533375), ('Country', 165.99832153320312, 0.28010015654287423, 0.14528851359441897), ('Age', 59.58226776123047, 0.10053717635790645, 0.05214883523917042), ('Area Income', 21.841712951660156, 0.036854994438608416, 0.0191167596141503), ('ID', 7.847520351409912, 0.013241650027549486, 0.006868470456372921), ('Male', 1.4073975086212158, 0.002374796677712208, 0.0012318118049354186)]}


In [64]:
# DeepLearning

mod,mod_id=get_model_by_algo("Dee",best_models)


In [65]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass       

{'algo': 'deeplearning', 'model_id': 'DeepLearning_0_AutoML_20181003_004019', 'varimp': [('Daily Internet Usage', 1.0, 1.0, 0.008321320696195242), ('Daily Time Spent on Site', 0.8003244400024414, 0.8003244400024414, 0.006659756326263183), ('Country.Uzbekistan', 0.7171310782432556, 0.7171310782432556, 0.005967477683270413), ('Country.United States Virgin Islands', 0.7000733613967896, 0.7000733613967896, 0.005825534951046076), ('Country.Chile', 0.6889277100563049, 0.6889277100563049, 0.0057327884118739255), ('Country.Mauritius', 0.6792511940002441, 0.6792511940002441, 0.005652267018549562), ('Country.Poland', 0.6684250235557556, 0.6684250235557556, 0.005562178982369302), ('Country.Saint Kitts and Nevis', 0.6636869311332703, 0.6636869311332703, 0.0055227517958335885), ('Country.Cayman Islands', 0.6634112596511841, 0.6634112596511841, 0.005520457845024354), ('Country.El Salvador', 0.6631825566291809, 0.6631825566291809, 0.005518554733834077), ('Country.Spain', 0.660958468914032, 0.66095846

In [66]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass     

In [67]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass     

In [68]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass     

{'algo': 'glm', 'model_id': 'GLM_grid_0_AutoML_20181003_004019_model_0', 'coef': {'Intercept': 2.271571991258173, 'Country.Afghanistan': -0.02510697201765049, 'Country.Albania': 0.0014679399209840455, 'Country.Algeria': 0.008970683282663962, 'Country.American Samoa': 0.0013318054171748543, 'Country.Andorra': -0.0025492844293272957, 'Country.Angola': 0.005334354170619788, 'Country.Anguilla': 0.00019325618479988657, 'Country.Antarctica (the territory South of 60 deg S)': -0.0020832988249473, 'Country.Antigua and Barbuda': -0.019265231106838095, 'Country.Argentina': 0.0011317922173191988, 'Country.Armenia': -0.0021361320524666064, 'Country.Aruba': -0.0024135386775945063, 'Country.Australia': -0.0086913183867361, 'Country.Austria': -0.013510322064655265, 'Country.Azerbaijan': 0.001302038147624808, 'Country.Bahamas': -0.014281003483370198, 'Country.Bahrain': -0.021259392271349674, 'Country.Bangladesh': -0.007613760410310062, 'Country.Barbados': -0.0027385562179286843, 'Country.Belarus': -0.

In [69]:
predictions_df=predictions_test(mod_best,test,run_id)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [70]:
predictions_df.head()

predict
1.00036
0.0400471
1.01139
0.0410509
0.850641
0.160596
0.987219
0.984361
1.02102
0.0497158


In [71]:
predictions_df.describe()

Rows:88
Cols:1




,predict
type,real
mins,-0.05840354319490006
mean,0.5127620500590856
maxs,1.1183824794457997
sigma,0.4679150202119868
zeros,0
missing,0
0,1.0003586853191397
1,0.040047124652463136
2,1.0113947158781715


In [72]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)    


In [73]:
meta_data

{'X': ['ID',
  'Age',
  'Male',
  'Country',
  'Daily Time Spent on Site',
  'Area Income',
  'Daily Internet Usage',
  'Ad Topic Line',
  'City',
  'Timestamp'],
 'analysis': 0,
 'balance': False,
 'balance_threshold': 0.2,
 'classification': False,
 'end_time': 1538541959.764308,
 'execution_time': 341.21927285194397,
 'max_models': 9,
 'min_mem_size': 6,
 'mod_best': 'StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019',
 'mod_best_algo': 'stackedensemble',
 'model_execution_time': 339.1064040660858,
 'models': ['DeepLearning_0_AutoML_20181003_004019',
  'XRT_0_AutoML_20181003_004019',
  'GLM_grid_0_AutoML_20181003_004019_model_0',
  'GBM_grid_0_AutoML_20181003_004019_model_58',
  'DRF_0_AutoML_20181003_004019',
  'StackedEnsemble_BestOfFamily_0_AutoML_20181003_004019'],
 'nthreads': 1,
 'project': None,
 'run_id': 'FAMH0nY14',
 'run_time': 333,
 'scale': False,
 'start_time': 1538541618.5450351,
 'target': None,
 'variables': {'Ad Topic Line': 'string',
  'Age': 'int',
  'Area In

In [74]:
# Clean up
os.chdir(server_path)

In [75]:
h2o.cluster().shutdown()

H2O session _sid_80e8 closed.


**When availble for your models you will need to make the following plots: **

## Variable Importance Plots

The Variable Importance Plot graphs the VIP values for each X variable. 

https://www.researchgate.net/figure/Random-forest-variable-importance-plot-Variables-are-ranked-in-terms-of-importance-on_fig2_295097543



## Partial Dependence Plots

For models that include only numerical values, you can view a Partial Dependence Plot (PDP) for that model. This provides a graphical representation of the marginal effect of a variable on the class probability (classification) or response (regression).

https://www.kaggle.com/dansbecker/partial-dependence-plots



## Gains/Lift Charts

[Score: Gains/Lift Table](http://h2o-release.s3.amazonaws.com/h2o/master/1648/docs-website/userguide/scoregainslift.html)  

The Gains/Lift Table page uses predicted data to evaluate model performance. The accuracy of the classification model for a random sample is evaluated according to the results when the model is and is not used.

The Gains/Lift Table is particularly useful for direct marketing applications, for example. The gains/lift chart shows the effectiveness of the current model(s) compared to a baseline, allowing users to quickly identify the most useful model.

To create a Gains/Lift table, H2O applies the model to each entry in the original dataset to find the response probability (Pi^), then orders the entries according to their predicted response probabilities. Finally, H2O divides the dataset into equal groups and calculates the average response rate for each group.

H2O uses the response rate of the top ten groups to evaluate the model performance; the highest response and greatest variation rates indicate the best model.

The lift is calculated from the gains. H2O uses the following formula to calculate the lift: λk=rkr⎯⎯⎯

where λk is the lift for k, rk is the response rate for k, and r⎯⎯⎯ is the average response rate for k. In other words, λk defines how much more likely k customers are to respond in comparison to the average response rate.

Requirements:

The vactual column must contain actual binary class labels.
The vpredict column must contain probabilities.

Evaluating Classifiers: Gains and Lift Charts https://youtu.be/1dYOcDaDJLY 

Understanding And Interpreting Gain And Lift Charts

   https://www.datasciencecentral.com/profiles/blogs/understanding-and-interpreting-gain-and-lift-charts
   
   

**Regression Metrics**

MSE

Mean Squared Error. The “squared” bit means the bigger the error, the more it is punished. 

deviance

Actually short for mean residual deviance. If the distribution is gaussian, then it is equal to MSE, and when not it usually gives a more useful estimate of error, which is why it is the default. Needs to be specified as “residual_deviance” when sorting grids.


RMSE
The square root of MSE. If your response variable units are dollars, the units of MSE is dollars-squared, but RMSE is back into dollars.

MAE
Mean Absolute Error.

R2
R-squared, also written as R², and also known as the coefficient of determination. 

RMSLE

Root Mean Squared Logarithmic Error. Prefer this to RMSE if an under-prediction is worse than an over-prediction.
